# CAMVID

In [1]:
!wget http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/data/LabeledApproved_full.zip

--2019-07-18 12:00:28--  http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/data/LabeledApproved_full.zip
Resolving mi.eng.cam.ac.uk (mi.eng.cam.ac.uk)... 129.169.82.147
Connecting to mi.eng.cam.ac.uk (mi.eng.cam.ac.uk)|129.169.82.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16567585 (16M) [application/zip]
Saving to: ‘LabeledApproved_full.zip’

LabeledApproved_ful 100%[===================>]  15.80M  3.01MB/s    in 6.7s    

2019-07-18 12:00:36 (2.35 MB/s) - ‘LabeledApproved_full.zip’ saved [16567585/16567585]



In [2]:
!unzip LabeledApproved_full.zip -d camvid

Archive:  LabeledApproved_full.zip
  inflating: camvid/0001TP_006690_L.png  
  inflating: camvid/0001TP_006720_L.png  
  inflating: camvid/0001TP_006750_L.png  
  inflating: camvid/0001TP_006780_L.png  
  inflating: camvid/0001TP_006810_L.png  
  inflating: camvid/0001TP_006840_L.png  
  inflating: camvid/0001TP_006870_L.png  
  inflating: camvid/0001TP_006900_L.png  
  inflating: camvid/0001TP_006930_L.png  
  inflating: camvid/0001TP_006960_L.png  
  inflating: camvid/0001TP_006990_L.png  
  inflating: camvid/0001TP_007020_L.png  
  inflating: camvid/0001TP_007050_L.png  
  inflating: camvid/0001TP_007080_L.png  
  inflating: camvid/0001TP_007110_L.png  
  inflating: camvid/0001TP_007140_L.png  
  inflating: camvid/0001TP_007170_L.png  
  inflating: camvid/0001TP_007200_L.png  
  inflating: camvid/0001TP_007230_L.png  
  inflating: camvid/0001TP_007260_L.png  
  inflating: camvid/0001TP_007290_L.png  
  inflating: camvid/0001TP_007320_L.png  
  inflating: camvid/0001TP_007350_L.png  

In [3]:
import os
len(os.listdir('camvid'))

701

In [0]:

!wget http://web4.cs.ucl.ac.uk/staff/g.brostow/MotionSegRecData/files/701_StillsRaw_full.zip

--2019-07-18 12:00:40--  http://web4.cs.ucl.ac.uk/staff/g.brostow/MotionSegRecData/files/701_StillsRaw_full.zip
Resolving web4.cs.ucl.ac.uk (web4.cs.ucl.ac.uk)... 128.16.6.5
Connecting to web4.cs.ucl.ac.uk (web4.cs.ucl.ac.uk)|128.16.6.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 584553911 (557M) [application/zip]
Saving to: ‘701_StillsRaw_full.zip’

701_StillsRaw_full.  68%[============>       ] 382.38M   124KB/s    eta 23m 45s

In [0]:
!wget http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/data/label_colors.txt

In [0]:
classes = []
colors = []
with open('label_colors.txt', 'rt') as f:
  for line in f:
    line = line.split()
    classes.append(line[3])
    colors.append((line[0], line[1], line[2]))

In [0]:
n_classes = len(classes)

In [0]:
# Split train test


# Model
__U-Net__

In [0]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (256,256,1), n_classes):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(n_classes, 1, activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model



# Train

In [0]:
# Data generator


In [0]:
model = unet(n_classes=n_classes)
model_checkpoint = ModelCheckpoint('unet_camvid.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,
                    steps_per_epoch=2000,
                    epochs=5,
                    callbacks=[model_checkpoint])